In [3]:
import numpy as np
import nbimporter
import utilities as utils
from collections import defaultdict
import pymongo

# Query
Example of a generalized query class which exploits the MongoDB index for different kinds of queries.

In [56]:
class Query(object):
    
    def __init__(self, dbname, index, language=('en', 'english'), host='localhost'):
        self.db = pymongo.MongoClient(host=host)[dbname]
        self.I = self.db[index]
        self.tokenizer = utils.Tokenizer(language)
        
        
    def boolean_ranking(self, query, norm='text'):
        """
        For each document, returns the number of query words therein contained
        """
        tokens = self.tokenizer.tokenize(0, query)
        g = {'$group': {'_id': {'token': '$' + norm, 'doc': '$document'}, 
                        'count': {'$sum': 1}}}
        m = {'$match': {norm: {'$in': [token[norm] for token in tokens]}}}
        k = {'$group': {'_id': '$_id.doc', 'relevance': {'$sum': 1}}}
        s = {'$sort': {'relevance': -1}}
        return [(r['_id'], r['relevance']) for r in self.I.aggregate([m, g, k, s])]